In [1]:

import pandas as pd

xlsx_path=r"C:\Users\user\Desktop\暑假語言模型\RAG資料集\test.xlsx"
csv_path=r"C:\Users\user\Desktop\暑假語言模型\RAG資料集\test.csv"

df=pd.read_excel(xlsx_path)
df.to_csv(csv_path,index=False,encoding="utf-8-sig")


In [2]:
df.head()

,Index,Name,Email
0,1,Sheryl,1@gmail.com
1,2,Preston,2@gmail.com
2,3,Roy,3@gmail.com
3,4,Linda,4@gmail.com
4,5,Joanna,5@gmail.com


In [3]:
from langchain_community.document_loaders import CSVLoader
loader=CSVLoader(csv_path)
docs=loader.load_and_split() #預設每一 row 直接變成一個 Document

c:\Users\user\anaconda3\envs\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_ollama import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

#embedding
embedding_model=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")

#vectorstore

vectorstore=Chroma.from_documents(embedding=embedding_model,documents=docs)
#retriever
retriever=vectorstore.as_retriever()

llm=ChatOllama(model="llama3.2:3b-instruct-q8_0",temperature=0)


In [12]:
def retrieve_context_per_question(question, chunks_query_retriever):
    """
    Retrieves relevant context and unique URLs for a given question using the chunks query retriever.

    Args:
        question: The question for which to retrieve context and URLs.

    Returns:
        A tuple containing:
        - A string with the concatenated content of relevant documents.
        - A list of unique URLs from the metadata of the relevant documents.
    """

    # Retrieve relevant documents for the given question
    docs = chunks_query_retriever.invoke(question, k=2)

    # Concatenate document content
    # context = " ".join(doc.page_content for doc in docs)
    context = [doc.page_content for doc in docs]

    return context

def show_context(context):
    """
    Display the contents of the provided context list.

    Args: context (list): A list of context items to be displayed.

    Prints each context item in the list with a heading indicating its position.
    """
    for i, c in enumerate(context):
        print(f"Context {i + 1}:")
        print(c)
        print("\n")

In [13]:
question="What is the  Roy email?"
context = retrieve_context_per_question(question, retriever)
show_context(context)

Context 1:
﻿Index: 3
Name: Roy
Email: 3@gmail.com


Context 2:
﻿Index: 1
Name: Sheryl
Email: 1@gmail.com




In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter


template="""
You are an assistant for question-answering tasks \n
Use the following text to answer \n
If you don't know the answer, say that you don't know \n\n

{context}

Question:{question}
"""
prompt=ChatPromptTemplate.from_template(template)



RAG_chain=(
    prompt
    | llm
    | StrOutputParser()
)

RAG_chain.invoke({"context":context,"question":question})

'The email for Roy is 3@gmail.com.'